In [3]:
import os
import polars as pl
import polars.selectors as cs
import pandas as pd
from datetime import date, time
import numpy as np
import dask.dataframe as dd
import dask.distributed
import pyarrow.parquet as pq




In [10]:
from azure.storage.blob import BlobServiceClient
import os
import pandas as pd
from io import BytesIO
from dotenv import load_dotenv
# Cargar variables de entorno desde el archivo .env
load_dotenv()

# Conectarse al servicio de Blob Storage
account_name = os.getenv("ACCOUNT_NAME")
account_key = os.getenv("ACCOUNT_KEY")
container_name = 'getdatalakefiles'

blob_service_client = BlobServiceClient(account_url=f"https://{account_name}.blob.core.windows.net", credential=account_key)

In [11]:
def list_parquet_files(blob_service_client, container_name, path):
    """
    Lista los archivos parquet en la ruta especificada dentro de un contenedor.

    Parameters:
    - blob_service_client (BlobServiceClient): El cliente del servicio Blob.
    - container_name (str): El nombre del contenedor.
    - path (str): La ruta dentro del contenedor.

    Returns:
    - list: Lista de nombres de archivos parquet en la ruta especificada.
    """
    # Obtener el cliente del contenedor
    container_client = blob_service_client.get_container_client(container_name)
    
    # Listar los blobs en la ruta especificada con extensión ".parquet"
    blobs = container_client.list_blobs(name_starts_with=path, include=['metadata'])
    
    # Filtrar los blobs con extensión ".parquet" y obtener sus nombres
    parquet_files = [blob.name for blob in blobs if blob.name.endswith('.parquet')]
    
    return parquet_files

# Ejemplo de uso:
# Reemplaza 'blob_service_client' y 'container_name' con tus propias variables
# Reemplaza 'path' con la ruta deseada dentro del contenedor
parquet_files = list_parquet_files(blob_service_client, container_name, "raw-data/yellow_analytics/")


In [12]:
parquet_files

['raw-data/yellow_analytics/yellow_analytics_part_10_1.parquet',
 'raw-data/yellow_analytics/yellow_analytics_part_10_2.parquet',
 'raw-data/yellow_analytics/yellow_analytics_part_10_3.parquet',
 'raw-data/yellow_analytics/yellow_analytics_part_10_4.parquet',
 'raw-data/yellow_analytics/yellow_analytics_part_11_1.parquet',
 'raw-data/yellow_analytics/yellow_analytics_part_11_2.parquet',
 'raw-data/yellow_analytics/yellow_analytics_part_11_3.parquet',
 'raw-data/yellow_analytics/yellow_analytics_part_11_4.parquet',
 'raw-data/yellow_analytics/yellow_analytics_part_12_1.parquet',
 'raw-data/yellow_analytics/yellow_analytics_part_12_2.parquet',
 'raw-data/yellow_analytics/yellow_analytics_part_12_3.parquet',
 'raw-data/yellow_analytics/yellow_analytics_part_12_4.parquet',
 'raw-data/yellow_analytics/yellow_analytics_part_13_1.parquet',
 'raw-data/yellow_analytics/yellow_analytics_part_13_2.parquet',
 'raw-data/yellow_analytics/yellow_analytics_part_13_3.parquet',
 'raw-data/yellow_analyti

In [13]:
# Obtener el cliente del contenedor y el blob
container_client = blob_service_client.get_container_client(container_name)
blob_client = container_client.get_blob_client("raw-data/yellow_analytics/yellow_analytics_part_1_1.parquet")

# Descargar el archivo desde Azure Blob Storage
downloaded_blob = blob_client.download_blob()

# Leer el archivo parquet en un DataFrame de pandas
with BytesIO() as file_stream:
    downloaded_blob.readinto(file_stream)
    file_stream.seek(0)
    df = pd.read_parquet(file_stream)


In [9]:
df.head()

,IdProveedor,TotalPasajeros,DistanciaViaje,IdTipoTarifa,IdZonaOrigen,IdZonaDestino,IdTipoPago,CostoTotal,FechaRecogida,HoraRecogida,DuracionViaje
1574,2,4,7.22,4.0,132,265,1,30.00,2022-01-01,0,718
1973,2,1,2.74,5.0,244,41,1,16.80,2022-01-01,0,855
2417,2,1,1.27,5.0,90,170,1,33.36,2022-01-01,0,592
3192,2,1,3.26,5.0,158,142,1,33.96,2022-01-01,0,732
3440,2,1,0.76,5.0,148,144,1,22.25,2022-01-01,0,371


In [14]:
def read_parquet_files_to_dataframe(blob_service_client, container_name, parquet_files):
    """
    Lee los archivos parquet y los une en un DataFrame.

    Parameters:
    - blob_service_client (BlobServiceClient): El cliente del servicio Blob.
    - container_name (str): El nombre del contenedor.
    - parquet_files (list): Lista de nombres de archivos parquet en el contenedor.

    Returns:
    - pd.DataFrame: DataFrame que contiene todos los datos de los archivos parquet.
    """
    dfs = []

    # Obtener el cliente del contenedor
    container_client = blob_service_client.get_container_client(container_name)

    for file_name in parquet_files:
        # Obtener el cliente del blob
        blob_client = container_client.get_blob_client(file_name)

        # Descargar el archivo desde Azure Blob Storage
        downloaded_blob = blob_client.download_blob()

        # Leer el archivo parquet en un DataFrame de pandas
        with BytesIO() as file_stream:
            downloaded_blob.readinto(file_stream)
            file_stream.seek(0)
            df = pd.read_parquet(file_stream)
            dfs.append(df)

    # Concatenar todos los DataFrames en uno solo
    result_df = pd.concat(dfs, ignore_index=True)
    return result_df

# Ejemplo de uso:
# Reemplaza 'blob_service_client' y 'container_name' con tus propias variables
# Reemplaza 'parquet_files' con la lista de nombres de archivos parquet
df = read_parquet_files_to_dataframe(blob_service_client, container_name, parquet_files)

In [16]:
df.shape

(39308, 11)

In [19]:
# Obtener la lista de archivos Parquet procesados en el directorio
directorio = "../datasets/processed/yellow_analytics/yellow_analytics_part_1_2.parquet"

# Lee el archivo Parquet utilizando Dask
df = pd.read_parquet(directorio)
#df_dask_particionado = df_dask.repartition(npartitions=4)
print(df.shape)
df.head()

(308, 11)


,IdProveedor,TotalPasajeros,DistanciaViaje,IdTipoTarifa,IdZonaOrigen,IdZonaDestino,IdTipoPago,CostoTotal,FechaRecogida,HoraRecogida,DuracionViaje
616017,2,4,2.29,5.0,161,68,1,27.80,2022-01-09,18,719
616032,2,4,1.23,5.0,246,230,1,39.36,2022-01-09,18,377
618730,1,1,0.90,5.0,75,263,1,35.20,2022-01-09,19,372
618826,1,1,8.10,5.0,138,130,2,46.75,2022-01-09,19,1075
619216,2,1,12.90,4.0,132,265,1,31.01,2022-01-09,19,941


In [ ]:
print(df.head(5))

In [23]:
# Obtener la lista de archivos Parquet procesados en el directorio
directorio = "../datasets/processed/ffvh_analytics/ffvh_analytics_part_0_2.parquet"

# Lee el archivo Parquet utilizando Dask
df = pd.read_parquet(directorio)
#df_dask_particionado = df_dask.repartition(npartitions=4)
print(df.shape)
df.head()

(2672926, 17)


,IdApp,IdProveedor,IdZonaOrigen,IdZonaDestino,DistanciaViaje,trip_time,TarifaPasajero,CostoTotal,FechaRecogida,HoraRecogida,FechaLlegada,HoraLlegada,FechaSolicitada,HoraSolicitada,FechaAtendida,HoraAtendida,DuracionAtencion
3687898,HV0003,B03404,232,26,8.33,1365,18.20,45.67,2022-01-09,12:18,2022-01-09,12:41,2022-01-09,12:12,2022-01-09,12:17,343.0
3687900,HV0003,B03404,145,14,14.07,1441,33.86,74.37,2022-01-09,12:40,2022-01-09,13:04,2022-01-09,12:39,2022-01-09,12:40,69.0
3687901,HV0003,B03404,49,17,1.17,386,7.91,14.94,2022-01-09,12:44,2022-01-09,12:50,2022-01-09,12:38,2022-01-09,12:42,227.0
3687902,HV0003,B03404,17,17,0.72,317,7.91,14.95,2022-01-09,12:58,2022-01-09,13:03,2022-01-09,12:54,2022-01-09,12:56,122.0
3687903,HV0003,B03404,138,41,7.04,921,20.97,50.24,2022-01-09,12:22,2022-01-09,12:37,2022-01-09,12:18,2022-01-09,12:21,180.0


In [ ]:
import json

# Convertir DataFrame a formato JSON compatible con Next.js y guardarlo en un archivo
df.to_json('tripsData.json', orient="records", lines=True)

# Leer el contenido del archivo JSON línea por línea
with open('tripsData.json', 'r') as f:
    lines = f.readlines()

# Convertir cada línea a un diccionario
data = [json.loads(line) for line in lines]

# Ahora data es una lista de diccionarios
print(data)



In [ ]:
# Obtener la lista de archivos Parquet procesados en el directorio
directorio = "../datasets/processed/ffvh_analytics/ffvh_analytics_part_0_1.parquet"

# Lee el archivo Parquet utilizando Dask
df = dd.read_parquet(directorio)
#df_dask_particionado = df_dask.repartition(npartitions=4)
df.head()